In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/handwriting-recognition/written_name_train_v2.csv")

print(max([len(i) for i in df.IDENTITY if type(i) is not float]))

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import random

In [ ]:
from torch.utils.data import Dataset, DataLoader
import cv2
class DataSet(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __getitem__(self, i):        
        x, y = self.df.iloc[i]
        
        if type(y) == float:
            return random.choice(self)
        
        res = []

        for l in y:
            l_encoded = ord(l.lower()) - ord('a')
            if l_encoded < 0 or l_encoded > 26:
                continue
            res.append(l_encoded)
        res.append(27)
        
        while len(res) < 35:
            res.append(28)
        
        return cv2.resize(cv2.imread("/kaggle/input/handwriting-recognition/train_v2/train/" + x, cv2.IMREAD_GRAYSCALE), (224, 224)), torch.tensor(res)

    def __len__(self):
        return len(self.df)

In [ ]:
train_ds = DataSet(df)
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5, 2)
        self.conv3 = nn.Conv2d(32, 64, 5, 2)
        self.conv4 = nn.Conv2d(64, 16, 5, 2)
        self.fc1 = nn.Linear(16*24*24, 512)
        self.rnn = nn.GRU(512, 512, 2)
        self.fc2 = nn.Linear(512, 28)
        self.softmax = nn.Softmax(2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        h = torch.zeros(2, x.shape[0], 512).cuda()
        x = x.unsqueeze(0)
        outputs = []
        
        for i in range(35):
            x, h = self.rnn(x, h)
            output = self.softmax(self.fc2(F.relu(x)))
            outputs.append(output[0])
            
        return torch.stack(outputs)


net = Net().cuda()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss(ignore_index=28)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
while True:
    losses = []
    accs = []
    for x, y in train_dl:
        x = x.float().unsqueeze(1).cuda()       
        y_hat = net(x).transpose(0, 1).cuda()
        
        #print(y_hat.shape, y.shape, y)
        
        y_hat = y_hat.reshape(-1, y_hat.shape[-1])
        y = y.reshape(-1).cuda()
        
        loss = criterion(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        # compute accuracy
        pred = torch.argmax(y_hat, 1)
        tot = 0
        cnt = 0
        for i in range(y.shape[0]):
            if y[i] == 28:
                continue
            if y[i] == pred[i]:
                cnt += 1
            tot += 1
        acc = cnt / tot
        accs.append(acc)
        
        if len(losses) % 100 == 0:
            print(np.mean(losses[-100:]), np.mean(accs[-100:]))